In [1]:
import pandas as pd

In [ ]:
# 데이터 호출
weather = pd.read_csv("../initial_weather_data/weather_data.csv")

# 타임라인 중복 제거
weather = weather.drop_duplicates(subset=["timestamp"])

In [ ]:
weather_check = weather.copy()

In [ ]:
## 빠진 타임 라인 찾기 ##

# timestamp를 datetime 형식으로 변환
weather_check['timestamp'] = pd.to_datetime(weather_check['timestamp'], format='%Y%m%d%H%M')

# 전체 기간 생성 (2024년 1월 1일부터 12월 31일까지 1시간 단위)
full_range = pd.date_range(start='2024-01-01 00:00', end='2024-12-31 23:00', freq='H')

# 빠진 timestamp 찾기
missing_timestamps = full_range.difference(weather_check['timestamp'])

# 결과 출력
print("빠진 timestamp 개수:", len(missing_timestamps))
print(missing_timestamps)

In [ ]:
import requests
import time

## 비어있는 타임라인 API 호출을 통해 테이터 수집 ##

# === 1. 기존 데이터 불러오기 ===
weather = pd.read_csv('./weather/weather_data.csv')
weather['timestamp'] = pd.to_datetime(weather['timestamp'], format='%Y%m%d%H%M')

# === 2. 빠진 timestamp 확보 ===
full_range = pd.date_range(start='2024-01-01 00:00', end='2024-12-31 23:00', freq='H')
missing_timestamps = full_range.difference(weather['timestamp'])

# === 3. Open API를 이용해 데이터 수집 ===
url = 'http://apis.data.go.kr/1360000/AirInfoService/getAirInfo'

new_data = []

for ts in missing_timestamps:
    fctm = ts.strftime('%Y%m%d%H%M')  # API 요청에 맞는 시간 형식 변환
    params = {
        'numOfRows': '100',
        'pageNo': '1',
        'dataType': 'JSON',
        'fctm': fctm,
        'icaoCode': 'RKSI'
    }

    response = requests.get(url, params=params)
    
    if response.status_code == 200:
        try:
            data = response.json()
            items = data.get("response", {}).get("body", {}).get("items", {}).get("item", [])

            # 가져온 데이터가 있다면 추가
            if items:
                for item in items:
                    new_data.append({
                        'timestamp': pd.to_datetime(item['tmFc'], format='%Y%m%d%H%M'),
                        'wind_direction': int(item['wd']),
                        'wind_speed': int(item['ws']),
                        'temperature': int(item['ta']),
                        'pressure': int(item['qnh'])
                    })

        except requests.exceptions.JSONDecodeError as e:
            print(f"\n🚨 JSONDecodeError 발생! (timestamp: {fctm})")
            print("응답 상태 코드:", response.status_code)
            print("응답 내용:\n", response.text)  # 실제 응답 내용을 출력
            break  # 에러 발생 시 루프 중단

    else:
        print(f"API 요청 실패: {fctm} (Status Code: {response.status_code})")

    time.sleep(1)  # API 요청 간격 조정 (너무 빠르게 호출하면 차단될 수도 있음)

# === 4. 새로운 데이터 추가 및 정렬 ===
new_df = pd.DataFrame(new_data)
weather_fixed = pd.concat([weather, new_df]).sort_values(by='timestamp').reset_index(drop=True)

# === 5. CSV로 저장 ===
weather_fixed.to_csv('./weather/weather_data_fixed.csv', index=False)

print("비어있던 timestamp 데이터 수집 및 보정 완료! 🚀")
